# Get-started (Local)
This guide will walk you through the main user journey of prompt flow code-first experience. You will learn how to create and develop your first prompt flow, test and evaluate it, then deploy it to production.

## Step 1: Create and develop a new bulk run

In [ ]:
import os
import promptflow.azure as pf 
from promptflow.azure.entities import BulkFlowRunInput

# configure azureml workspace ml_client
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="<subscription-id>",
    resource_group_name="<resource-group-name>",
    workspace_name="<workspace-name>",
)
pf.configure(client=client)



In [ ]:
# load flow
flow_dir = "../../flows/standard/web-classification"
data = "../../flows/standard/web-classification/data.jsonl"
runtime = "<runtime-name>"

flow = pf.load_flow(flow_dir)
print(flow)

### Add connection
Since flow `web-classification` uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before.

Please go to workspace portal, click `Prompt flow` -> `Connections` -> `Create`, then follow the instruction to create your own connections.

### Submit bulk run

In [ ]:
# standard flow using remote data
run = flow.submit_bulk_run(
    data=data,
    connections="<connection-name>",  # To be confirmed.
    runtime=runtime,
)
print(run)

In [ ]:
run.wait_for_completion()

In [ ]:
pf.show_details(run)

In [ ]:
pf.show_metrics(run)

## Step 2: Evaluate your bulk flow run

In [ ]:
from promptflow.azure.entities import BulkFlowRunInput


classification_accuracy_eval = pf.load_flow("../../flows/standard/classification_accuracy_evaluation")

bulk_flow_run_input = BulkFlowRunInput(
    data=data,
    variants=[run],
    inputs_mapping={"groundtruth": "${data.answer}","prediction": "${variant.outputs.category}"},
)

baseline_accuracy = classification_accuracy_eval.submit_bulk_run(
    data=bulk_flow_run_input,
    runtime=runtime,
)

print(baseline_accuracy)

In [ ]:
baseline_accuracy.wait_for_completion()

In [ ]:
pf.show_details(baseline_accuracy)

In [ ]:
pf.show_metrics(baseline_accuracy)

## Step 3: Deploy the flow (WIP)